In [1]:
import os
import sys

import e3nn_jax as e3nn
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import pickle

# Add src directory to path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
src_dir = os.path.join(project_root, 'src')
sys.path.append(src_dir)

from utils.plotters import visualize_geometry, colorplot, visualize_signal, compare_geometries
from utils.geometries import trigonal_plane, tetrahedron, octahedron, icosahedron, dodecahedron
from utils.alignment import sample_uniform_quaternion, align_signals, find_best_random_quaternion, rotate_points_quaternion, stack_points, point_distance, spherical_harmonic_distance, stack_points, choose_best_quaternion, evenly_distributed_quaternions, quaternion_rotation_distance, spherical_grid_distance
import spectra
from spectra import sum_of_diracs, powerspectrum, bispectrum, trispectrum, invert, reconstruct_geometry

# Enable 64-bit precision
jax.config.update("jax_enable_x64", True)

In [1]:
import numpy as np

def extract_multiple_structures(filename, n_structures=None):
    """
    Extract coordinates from multiple structures in a file.
    
    Parameters:
    -----------
    filename : str
        Path to the input file
    n_structures : int or None
        Number of structures to extract. If None, extract all structures.
        
    Returns:
    --------
    list of numpy.ndarray
        List of coordinate arrays, each with shape (8, 3)
    """
    structures = []
    
    with open(filename, 'r') as file:
        lines = file.readlines()
        
        # Calculate total number of structures in the file
        total_structures = len(lines) // 10
        
        # If n_structures is not specified or is larger than available,
        # use all available structures
        if n_structures is None or n_structures > total_structures:
            n_structures = total_structures
        
        # Process each structure
        for struct_idx in range(n_structures):
            # Calculate the starting line for this structure
            start_line = struct_idx * 10
            
            # Read the number of atoms (should be 8 based on your example)
            n_atoms = int(lines[start_line].strip())
            
            # Skip the header line and extract coordinates
            coordinates = []
            for i in range(start_line + 2, start_line + 2 + n_atoms):
                line = lines[i].strip().split()
                # Extract the x, y, z coordinates
                x, y, z = float(line[1]), float(line[2]), float(line[3])
                coordinates.append([x, y, z])
            
            structures.append(np.array(coordinates))
    
    return structures

In [2]:
# Example usage
filename = "../data/boron_8000_pbeccpvdz.xyz"

# Extract the first 5 structures
structures = extract_multiple_structures(filename, n_structures=5)

# Print information about the extracted structures
print(f"Extracted {len(structures)} structures")
for i, coords in enumerate(structures):
    print(f"Structure {i+1}: shape {coords.shape}")
    
# You can access individual structures like this:
first_structure = structures[0]
print(first_structure)  # Prints the 8x3 array of coordinates for the first structure

Extracted 5 structures
Structure 1: shape (8, 3)
Structure 2: shape (8, 3)
Structure 3: shape (8, 3)
Structure 4: shape (8, 3)
Structure 5: shape (8, 3)
[[ 0.          0.          0.        ]
 [ 1.39454902  0.          0.57793632]
 [-1.23980586  0.63847348  0.57793632]
 [-0.29503787 -1.36298188  0.57793632]
 [-0.91652028  1.05107447 -0.57793632]
 [ 0.33360178 -1.35405938 -0.57793632]
 [ 1.22118593  0.6734032  -0.57793632]
 [ 0.          0.          1.76852686]]


In [ ]:
from tqdm import tqdm

lmax = 5
predicted_geometries = []
avg_errors = []
max_errors = []

for true_geometry in tqdm(local_envs):
    true_geometry = jnp.array(true_geometry)
    true_signal = sum_of_diracs(true_geometry, lmax)
    true_bispectrum = bispectrum(true_signal)
    predicted_geometry = reconstruct_geometry(true_bispectrum, bispectrum, lmax, true_geometry)
    predicted_geometries.append(predicted_geometry)
    total_point_distance_error, max_point_distance_error = point_distance(predicted_geometry, true_geometry)
    avg_point_distance_error = total_point_distance_error / len(predicted_geometry)
    avg_errors.append(avg_point_distance_error)
    max_errors.append(max_point_distance_error)

  0%|          | 0/20 [00:00<?, ?it/s]

step 0, loss: 0.2171665902423155
step 100, loss: 0.00762747050447586
step 200, loss: 0.00022933883304288017
step 300, loss: 4.806759365545818e-05
step 400, loss: 0.0001870952434313177
step 500, loss: 0.0002192004268960184
Stopping early at step 505: no improvement for 100 iterations
Final best loss: 1.1378633520977924e-05
step 0, loss: 0.0003568545363812495
step 100, loss: 2.444962235450216e-06
Stopping early at step 171: no improvement for 100 iterations
Final best loss: 7.287387205931345e-08
Early stopping at iteration 192 due to no improvement for 100 iterations


  5%|▌         | 1/20 [00:50<15:52, 50.11s/it]

step 0, loss: 0.21799429809417442
step 100, loss: 0.007060204642570919
step 200, loss: 0.0008972752173861331
step 300, loss: 0.0004083242133645475
step 400, loss: 0.0006081186292695111
Stopping early at step 478: no improvement for 100 iterations
Final best loss: 4.893811632390341e-05
step 0, loss: 0.00023215882929991508
step 100, loss: 8.002290999844348e-06
step 200, loss: 5.3676903236588815e-06
step 300, loss: 6.179496654849328e-06
step 400, loss: 5.011756684471862e-06
step 500, loss: 4.2754319730079905e-06
step 600, loss: 4.916906887470895e-06
Stopping early at step 692: no improvement for 100 iterations
Final best loss: 1.4452422853499063e-06


 10%|█         | 2/20 [01:04<08:43, 29.11s/it]

...

step 0, loss: 0.21789513288710116
step 100, loss: 0.008185372212533081
step 200, loss: 0.0004357071665694266
step 300, loss: 0.00010565272968020559
Stopping early at step 383: no improvement for 100 iterations
Final best loss: 4.756233287584436e-05
step 0, loss: 0.0002253544029709146
step 100, loss: 3.346830212079543e-05
step 200, loss: 4.011969602837526e-05
Stopping early at step 268: no improvement for 100 iterations
Final best loss: 3.254936994942319e-05


 70%|███████   | 14/20 [03:10<01:00, 10.06s/it]

step 0, loss: 0.22319972119293505
step 100, loss: 0.011346769741127558
step 200, loss: 0.0009361688945340854
step 300, loss: 0.00010301146300485658
step 400, loss: 0.00041211480832643415
step 500, loss: 0.00017487584328397863
Stopping early at step 539: no improvement for 100 iterations
Final best loss: 1.6489341243024585e-05
step 0, loss: 0.0004858687194310529
step 100, loss: 9.425783314961637e-07
step 200, loss: 3.5101466031501333e-07
Stopping early at step 283: no improvement for 100 iterations
Final best loss: 1.458625216396869e-07


 75%|███████▌  | 15/20 [03:19<00:48,  9.72s/it]

Early stopping at iteration 211 due to no improvement for 100 iterations
step 0, loss: 0.221692123328246
step 100, loss: 0.008701549244006707
step 200, loss: 0.0016733145638251823
step 300, loss: 0.00011379756579190932
step 400, loss: 0.0002269621129022284
step 500, loss: 0.0002963359052120507
Stopping early at step 521: no improvement for 100 iterations
Final best loss: 1.8555186464257674e-05
step 0, loss: 0.00020610971141038657
step 100, loss: 3.760166009879709e-07
step 200, loss: 3.1414052558316066e-06
Stopping early at step 272: no improvement for 100 iterations
Final best loss: 3.623643292055396e-08
